In [0]:
from google.colab import files
uploaded = files.upload()

import io
data = pd.read_csv(io.BytesIO(uploaded['census_data.csv']))
data.head()

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [15]:
path = "/content/drive/My Drive/census_data.csv"
data = pd.read_csv(path)
data.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [0]:

data=data.dropna()

In [0]:
data.income_bracket.replace(' <=50K',0,inplace=True)
data.income_bracket.replace(' >50K',1,inplace=True)

In [0]:
X=data.drop('income_bracket',axis=1)
y=data.income_bracket

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = .30, random_state = 102)

In [0]:
gender=tf.feature_column.categorical_column_with_vocabulary_list("gender",["Female","Male"])
occu=tf.feature_column.categorical_column_with_hash_bucket("occupation",hash_bucket_size=1000)
martial=tf.feature_column.categorical_column_with_hash_bucket("marital_status",hash_bucket_size=1000)
edu=tf.feature_column.categorical_column_with_hash_bucket("education",hash_bucket_size=1000)
rel=tf.feature_column.categorical_column_with_hash_bucket("relationship",hash_bucket_size=1000)
wc=tf.feature_column.categorical_column_with_hash_bucket("workclass",hash_bucket_size=1000)
nc=tf.feature_column.categorical_column_with_hash_bucket("native_country",hash_bucket_size=1000)

In [12]:
data.describe()

,age,education_num,capital_gain,capital_loss,hours_per_week,income_bracket
count,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,10.080679,1077.648844,87.303830,40.437456,0.240810
std,13.640433,2.572720,7385.292085,402.960219,12.347429,0.427581
min,17.000000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000,0.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000,0.000000
75%,48.000000,12.000000,0.000000,0.000000,45.000000,0.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000,1.000000


In [0]:
age=tf.feature_column.numeric_column("age")
edu_num=tf.feature_column.numeric_column("education_num")
gain=tf.feature_column.numeric_column("capital_gain")
loss=tf.feature_column.numeric_column("capital_loss")
hpw=tf.feature_column.numeric_column("hours_per_week")

In [0]:
feat_col=[gender,occu,martial,edu,rel,wc,nc,age,edu_num,gain,loss,hpw]

In [0]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=None,shuffle=True)

In [26]:
model=tf.estimator.LinearClassifier(feature_columns=feat_col)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpu1lckg2k', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5fa4cf53c8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [27]:
model.train(input_fn=input_func,steps=5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpu1lckg2k/mo

In [0]:
pred_fn=tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [29]:
predictions=list(model.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpu1lckg2k/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [30]:
predictions[0]

{'all_class_ids': array([0, 1], dtype=int32),
 'all_classes': array([b'0', b'1'], dtype=object),
 'class_ids': array([0]),
 'classes': array([b'0'], dtype=object),
 'logistic': array([0.00546741], dtype=float32),
 'logits': array([-5.203464], dtype=float32),
 'probabilities': array([0.9945326 , 0.00546743], dtype=float32)}

In [0]:
final_pred=[]
for pred in predictions:
    final_pred.append(pred['class_ids'][0])
    

In [32]:
final_pred[:10],y_test[:10]

([0, 1, 1, 0, 0, 0, 1, 1, 1, 0], 8822     0
 5275     1
 26101    1
 28147    0
 27101    0
 31984    0
 15119    1
 31452    1
 7681     1
 16391    0
 Name: income_bracket, dtype: int64)

In [33]:
print(classification_report(y_test,final_pred))

              precision    recall  f1-score   support

           0       0.90      0.86      0.88      7441
           1       0.62      0.71      0.66      2328

    accuracy                           0.83      9769
   macro avg       0.76      0.79      0.77      9769
weighted avg       0.84      0.83      0.83      9769

